!pip install librosa

!pip install demucs

!pip install diffq

In [ ]:
import os
import shutil
from moviepy.editor import VideoFileClip

#비디오를 길이가 짧은 순서대로 정렬한 후 '1.mp4', '2.mp4' ... 이름으로 다시 저장하고 각 비디오의 오디오 트랙을 1.wav, 2.wav...로 추출하여 저장하는 함수
def sort_videos_by_length_and_extract_audio(input_folder, output_audio_folder, output_video_folder):
    # input_folder에서 비디오 파일 리스트 가져오기
    video_files = [f for f in os.listdir(input_folder) if f.endswith(('.mp4', '.avi', '.mov'))]
    video_paths = [os.path.join(input_folder, f) for f in video_files]
    
    # 비디오 파일마다 길이 저장
    video_lengths = []
    for video_path in video_paths:
        with VideoFileClip(video_path) as video:
            video_length = video.duration
        video_lengths.append((video_path, video_length))
    
    # 비디오 길이에 따라 정렬
    sorted_videos = sorted(video_lengths, key=lambda x: x[1])
    
    # 정렬된 순서대로 파일 복사 및 이름 변경, 오디오 추출
    for i, (video_path, _) in enumerate(sorted_videos, start=1):
        # 새로운 파일 이름 설정
        new_video_name = f"{i}{os.path.splitext(video_path)[1]}"
        
        # 원본 비디오 파일을 새 위치로 복사
        new_video_path = os.path.join(output_video_folder, new_video_name)
        shutil.copy(video_path, new_video_path)
        
        # 오디오 추출
        with VideoFileClip(new_video_path) as video:
            audio_path = os.path.join(output_audio_folder, f"{i}.wav")
            video.audio.write_audiofile(audio_path)

In [ ]:
import librosa

def find_last_end_point(audio_path, threshold_diff=0.005, threshold_energy=-40):
    # 음악 파일 로드
    y, sr = librosa.load(audio_path)

    # 하모닉/퍼커시브 분리
    y_harmonic, y_percussive = librosa.effects.hpss(y)

    # 퍼커시브 부분의 스펙트로그램 생성
    D_percussive = librosa.stft(y_percussive)

    # 복소수 스펙트로그램을 실수로 변환
    D_percussive_magnitude = np.abs(D_percussive)

    # 스펙트럼의 변화 계산
    spectral_diff_percussive = np.abs(librosa.feature.delta(D_percussive_magnitude))

    # 에너지 계산
    energy = np.sum(librosa.amplitude_to_db(D_percussive_magnitude, ref=np.max), axis=0)

    # 변화가 급격한 부분 탐색
    end_points = []
    for i in range(1, spectral_diff_percussive.shape[1]):
        if spectral_diff_percussive[0, i-1] - spectral_diff_percussive[0, i] > threshold_diff and energy[i] < threshold_energy:
            end_point_sec = librosa.frames_to_time(i, sr=sr)
            end_points.append(end_point_sec)

    # 마지막 끝점을 반환
    return end_points[-1] + 2 #2초 더해준건 경험에 의함.

In [ ]:
from scipy.io import wavfile

def trim_audio_files(output_audio_folder, music_length):
    # 오디오 파일 목록 가져오기
    audio_files = [f for f in os.listdir(output_audio_folder) if f.endswith('.wav')]
    audio_paths = [os.path.join(output_audio_folder, f) for f in audio_files]
    
    # 각 오디오 파일에 대해 지정된 길이만큼 자르기
    for i, audio_path in enumerate(audio_paths):
        # 오디오 파일 읽기
        rate, audio_file = wavfile.read(audio_path)
        
        # 지정된 길이만큼 자르기 (밀리초를 샘플 수로 변환)
        desired_length_samples = int(music_length * rate)
        if desired_length_samples < len(audio_file):
            trimmed_audio = audio_file[:desired_length_samples]
        else:
            trimmed_audio = audio_file
        
        # 결과 저장
        output_path = os.path.join(output_audio_folder, os.path.basename(audio_path))
        wavfile.write(output_path, rate, trimmed_audio)

In [ ]:
import numpy as np
from scipy.signal import correlate
from scipy.io import wavfile
import os

#기준 영상과 비교 영상의 오디오 싱크를 맞췄을 때 비교 영상의 앞부분에서 얼만큼의 시간을 조정해야 하는지를 계산하는 함수
def cal_adjustment_time(audio_path):
    # audio_path 내의 모든 wav 파일 리스트 가져오기
    audio_files = [f for f in os.listdir(audio_path) if f.endswith('.wav')]
    audio_paths = [os.path.join(audio_path, f) for f in audio_files]

    # 기준 영상 대비 비교 영상의 앞부분이 얼마만큼 밀려있는지를 저장하는 리스트
    front_adjustment_lst = []

    # 기준 영상의 길이 계산
    _, base_audio = wavfile.read(audio_paths[0])
    base_duration = len(base_audio) / 44100  # 기준 오디오 파일의 길이(초)

    # audio_path 내의 모든 target audio 파일에 대해 adjusted_front_duration 계산
    for audio_path in audio_paths[1:]:
        rate, target_audio = wavfile.read(audio_path)
        target_duration = len(target_audio) / rate  # target 오디오 파일의 길이(초)

        base_waveform = base_audio  # 기준 무대 영상의 waveform
        target_waveform = target_audio  # 비교할 무대 영상의 waveform

        # waveform 정규화
        base_waveform = base_waveform / np.max(np.abs(base_waveform))
        target_waveform = target_waveform / np.max(np.abs(target_waveform))

        # 상호 상관 함수 계산
        correlation = correlate(target_waveform, base_waveform, mode='same')

        # 상호 상관 함수에서 피크값 찾기
        peak_index = np.argmax(correlation)

        # base_audio와 target_audio간의 adjusted_front_duration 계산
        adjusted_front_duration = ((peak_index - len(base_waveform)) // 2) / rate
        front_adjustment_lst.append(adjusted_front_duration)

    return front_adjustment_lst

### CPU 병렬처리

In [ ]:
from trim_vids_and_extract_frames import trim_and_extract_frames_parallel

# 폴더 경로 설정
input_folder = 'input_videos'
output_audios_folder = 'output_audios' #길이순으로 정렬된 영상에서 추출한 오디오 트랙들이 저장되는 폴더
output_videos_folder = 'output_videos' #길이순으로 정렬된 영상이 새로운 이름으로 저장되는 폴더
output_frames_folder = 'output_frames' # 추출한 프레임들이 저장될 폴더

# 오디오 추출 및 오디오 길이에 따른 영상 정렬
sort_videos_by_length_and_extract_audio(input_folder, output_audios_folder, output_videos_folder) #input_folder에 있는 영상들을 길이순으로 정렬하고 오디오 트랙 추출 및 이름 변경하여 새롭게 저장

In [ ]:
!demucs --two-stems=vocals -n mdx_extra_q "output_audios/1.wav"

In [ ]:
# 프레임 추출 특성 설정
fps = 29.97 # 추출할 초당 프레임수 지정
resize_size = (1920, 1080)  # 이미지 리사이즈 크기
base_audio_path = 'separated/mdx_extra_q/1/no_vocals.wav'
music_length = find_last_end_point(base_audio_path)
trim_audio_files(output_audios_folder, music_length)

#프레임 추출 실행
front_adjustment_lst = cal_adjustment_time(output_audios_folder) #길이가 가장 짧은 무대 영상을 기준으로 했을 때 비교할 영상이 앞뒤로 얼마만큼 조정해야지를 계산
trim_and_extract_frames_parallel(output_videos_folder, output_frames_folder, front_adjustment_lst, resize_size, music_length, fps) #비교 영상이 기준 영상과 길이가 같고 오디오 싱크가 맞도록 조정한 후 프레임을 추출하여 저장